In [1]:
# %% 1. การติดตั้งและ Import Library

# ติดตั้ง Library ที่จำเป็น
!pip install -q transformers sentence-transformers datasets lightgbm scikit-learn

import pandas as pd
import numpy as np
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from tqdm.auto import tqdm

# ปิด warning ที่ไม่จำเป็นของ pandas
pd.options.mode.chained_assignment = None

print("ติดตั้งและ Import Library สำเร็จ")


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



ติดตั้งและ Import Library สำเร็จ


In [6]:
# %% 2. การเตรียมข้อมูล

print("กำลังโหลด STS-B Dataset เพื่อจำลองข้อมูลการแข่งขัน...")
# โหลดชุดข้อมูล STS-B
dataset = load_dataset("PhilipMay/stsb_multi_mt", "en")
train_data = dataset['train']
test_data = dataset['test']

# --- สร้างข้อมูลสำหรับฝึกซ้อม (train.csv) ---
train_df = pd.DataFrame({
    'resume_text': [item['sentence1'] for item in train_data],
    'job_description_text': [item['sentence2'] for item in train_data],
    'match_score': [item['similarity_score'] for item in train_data]
})

# --- สร้างข้อมูลสำหรับทดสอบ (test.csv) ---
test_df = pd.DataFrame({
    'resume_text': [item['sentence1'] for item in test_data],
    'job_description_text': [item['sentence2'] for item in test_data],
    # ในข้อมูลจริงจะไม่มี score เราจะเก็บไว้เพื่อประเมินผลโมเดลภายหลัง
    'actual_score': [item['similarity_score'] for item in test_data]
})
test_df['pair_id'] = test_df.index





กำลังโหลด STS-B Dataset เพื่อจำลองข้อมูลการแข่งขัน...


In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 5749
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1379
    })
    dev: Dataset({
        features: ['sentence1', 'sentence2', 'similarity_score'],
        num_rows: 1500
    })
})

In [4]:
train_df.head()

,resume_text,job_description_text,match_score
0,A plane is taking off.,An air plane is taking off.,5.00
1,A man is playing a large flute.,A man is playing a flute.,3.80
2,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,3.80
3,Three men are playing chess.,Two men are playing chess.,2.60
4,A man is playing the cello.,A man seated is playing the cello.,4.25


In [5]:
test_df.head()

,resume_text,job_description_text,actual_score,pair_id
0,A girl is styling her hair.,A girl is brushing her hair.,2.5,0
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,3.6,1
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,5.0,2
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,4.2,3
4,A man is playing a harp.,A man is playing a keyboard.,1.5,4


In [8]:
# %% 3. การโหลดโมเดล Bi-Encoder

# โหลดโมเดลที่ถูก pre-trained มาสำหรับงาน Semantic Similarity
# หมายเหตุ: หากทำงานกับภาษาไทยจริงๆ ควรเปลี่ยนเป็นโมเดลที่รองรับภาษาไทย
# เช่น 'airesearch/wangchanberta-base-att-spm-uncased' แล้วใช้ผ่าน Hugging Face
# paraphrase-multilingual-mpnet-base-v2 , sim-cse-th-best , phayathaibert
MODEL_NAME = 'all-MiniLM-L6-v2'
# bge-large-en-v1.5 , gte-large , e5-mistral-7b-instruct
print(f"กำลังโหลดโมเดล Bi-Encoder: '{MODEL_NAME}'...")
# SentenceTransformer จะจัดการเรื่อง device (GPU/CPU) ให้อัตโนมัติ
encoder_model = SentenceTransformer(MODEL_NAME)

print("โหลดโมเดลสำเร็จ")
print("ตัวอย่างโมเดล:", encoder_model)


กำลังโหลดโมเดล Bi-Encoder: 'all-MiniLM-L6-v2'...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\potij\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

โหลดโมเดลสำเร็จ
ตัวอย่างโมเดล: SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


In [9]:
# %% 4. การสร้าง Feature (Embedding และ Cosine Similarity)

def create_similarity_features(df, model):
    """
    ฟังก์ชันสำหรับสร้าง Embedding และคำนวณ Cosine Similarity
    """
    print("กำลังสร้าง Embedding สำหรับ Resume...")
    resume_embeddings = model.encode(df['resume_text'].tolist(), show_progress_bar=True, convert_to_tensor=True)

    print("\nกำลังสร้าง Embedding สำหรับ Job Description...")
    job_embeddings = model.encode(df['job_description_text'].tolist(), show_progress_bar=True, convert_to_tensor=True)

    print("\nกำลังคำนวณ Cosine Similarity...")
    # ย้าย embedding กลับมาที่ CPU เพื่อคำนวณด้วย numpy
    resume_embeddings_cpu = resume_embeddings.cpu().numpy()
    job_embeddings_cpu = job_embeddings.cpu().numpy()

    # คำนวณ Cosine Similarity (dot product ของ unit vectors)
    # เราจะคำนวณแบบ element-wise (แถวต่อแถว)
    dot_products = np.sum(resume_embeddings_cpu * job_embeddings_cpu, axis=1)
    norm_resume = np.linalg.norm(resume_embeddings_cpu, axis=1)
    norm_job = np.linalg.norm(job_embeddings_cpu, axis=1)

    # เพิ่มค่า epsilon เล็กน้อยเพื่อป้องกันการหารด้วยศูนย์
    epsilon = 1e-8
    cosine_similarities = dot_products / (norm_resume * norm_job + epsilon)

    df['cosine_similarity'] = cosine_similarities
    return df

# สร้างฟีเจอร์สำหรับข้อมูล Train
train_df_featured = create_similarity_features(train_df, encoder_model)

print("\nสร้างฟีเจอร์สำหรับ Train Set สำเร็จ")
print("ตัวอย่างข้อมูล Train พร้อมฟีเจอร์ใหม่:")
print(train_df_featured.head())


กำลังสร้าง Embedding สำหรับ Resume...


Batches:   0%|          | 0/180 [00:00<?, ?it/s]


กำลังสร้าง Embedding สำหรับ Job Description...


Batches:   0%|          | 0/180 [00:00<?, ?it/s]


กำลังคำนวณ Cosine Similarity...

สร้างฟีเจอร์สำหรับ Train Set สำเร็จ
ตัวอย่างข้อมูล Train พร้อมฟีเจอร์ใหม่:
                                     resume_text  \
0                         A plane is taking off.   
1                A man is playing a large flute.   
2  A man is spreading shreded cheese on a pizza.   
3                   Three men are playing chess.   
4                    A man is playing the cello.   

                                job_description_text  match_score  \
0                        An air plane is taking off.         5.00   
1                          A man is playing a flute.         3.80   
2  A man is spreading shredded cheese on an uncoo...         3.80   
3                         Two men are playing chess.         2.60   
4                 A man seated is playing the cello.         4.25   

   cosine_similarity  
0           0.939303  
1           0.902032  
2           0.892001  
3           0.794561  
4           0.928653  


In [10]:
train_df_featured

,resume_text,job_description_text,match_score,cosine_similarity
0,A plane is taking off.,An air plane is taking off.,5.00,0.939303
1,A man is playing a large flute.,A man is playing a flute.,3.80,0.902032
2,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,3.80,0.892001
3,Three men are playing chess.,Two men are playing chess.,2.60,0.794561
4,A man is playing the cello.,A man seated is playing the cello.,4.25,0.928653
...,...,...,...,...
5744,Severe Gales As Storm Clodagh Hits Britain,Merkel pledges NATO solidarity with Latvia,0.00,0.058639
5745,Dozens of Egyptians hostages taken by Libyan t...,Egyptian boat crash death toll rises as more b...,0.00,0.314323
5746,President heading to Bahrain,President Xi: China to continue help to fight ...,0.00,0.037030
5747,"China, India vow to further bilateral ties",China Scrambles to Reassure Jittery Stock Traders,0.00,0.302837


In [20]:
# %% 5. การฝึกโมเดล Regression (LightGBM)

# เตรียมข้อมูลสำหรับ LightGBM
# X คือฟีเจอร์ (Feature)
X_train = train_df_featured[['cosine_similarity']]
# y คือเป้าหมาย (Target)
y_train = train_df_featured['match_score']

# กำหนดพารามิเตอร์สำหรับ LightGBM
# เราใช้ 'regression' เป็น objective และ 'rmse' เป็น metric สำหรับการประเมิน
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'n_estimators': 2000,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'n_jobs': -1,
    'seed': 42
}

print("กำลังฝึกโมเดล LightGBM...")

# สร้างและฝึกโมเดล
regression_model = lgb.LGBMRegressor(**lgb_params)
regression_model.fit(X_train, y_train,
                     eval_set=[(X_train, y_train)],
                     eval_metric='rmse',
                     callbacks=[lgb.early_stopping(100, verbose=False)])

print("ฝึกโมเดล LightGBM สำเร็จ!")


กำลังฝึกโมเดล LightGBM...


c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\callback.py:341: UserWarning: Only training set found, disabling early stopping.
  _log_warning("Only training set found, disabling early stopping.")


ฝึกโมเดล LightGBM สำเร็จ!


In [21]:
# %% 6. การทำนายผลสำหรับ Test Set

print("กำลังสร้างฟีเจอร์สำหรับ Test Set...")
# 1. สร้างฟีเจอร์สำหรับ Test Set
test_df_featured = create_similarity_features(test_df, encoder_model)

print("\nสร้างฟีเจอร์สำหรับ Test Set สำเร็จ")


# 2. ทำนายผลด้วยโมเดล Regression ที่ฝึกแล้ว
print("กำลังทำนาย match_score สำหรับ Test Set...")
X_test = test_df_featured[['cosine_similarity']]
predictions = regression_model.predict(X_test)

# เพิ่มผลลัพธ์การทำนายลงใน DataFrame
test_df_featured['predicted_score'] = predictions

print("ทำนายผลสำเร็จ")
print("ตัวอย่างผลการทำนาย:")
# แสดง 'pair_id', 'predicted_score', และ 'actual_score' เพื่อเปรียบเทียบ
print(test_df_featured[['pair_id', 'predicted_score', 'actual_score']].head())

# ประเมินผลเทียบกับคำตอบจริง (ขั้นตอนนี้ทำได้เพราะเรามี 'actual_score')
rmse = np.sqrt(mean_squared_error(test_df_featured['actual_score'], test_df_featured['predicted_score']))
print(f"\nRoot Mean Squared Error (RMSE) บน Test Set: {rmse:.4f}")


กำลังสร้างฟีเจอร์สำหรับ Test Set...
กำลังสร้าง Embedding สำหรับ Resume...


Batches:   0%|          | 0/44 [00:00<?, ?it/s]


กำลังสร้าง Embedding สำหรับ Job Description...


Batches:   0%|          | 0/44 [00:00<?, ?it/s]


กำลังคำนวณ Cosine Similarity...

สร้างฟีเจอร์สำหรับ Test Set สำเร็จ
กำลังทำนาย match_score สำหรับ Test Set...
ทำนายผลสำเร็จ
ตัวอย่างผลการทำนาย:
   pair_id  predicted_score  actual_score
0        0         3.533265           2.5
1        1         3.408280           3.6
2        2         4.168404           5.0
3        3         3.634500           4.2
4        4         1.191888           1.5

Root Mean Squared Error (RMSE) บน Test Set: 0.8645


In [24]:
print(test_df_featured[['pair_id', 'predicted_score', 'actual_score']].head(20))


    pair_id  predicted_score  actual_score
0         0         3.533265         2.500
1         1         3.408280         3.600
2         2         4.168404         5.000
3         3         3.634500         4.200
4         4         1.191888         1.500
5         5         1.947011         1.800
6         6         3.603353         3.500
7         7         1.898054         2.200
8         8         2.322161         2.200
9         9         2.008990         1.714
10       10         2.008990         1.714
11       11         4.609476         5.000
12       12         1.257835         0.600
13       13         3.668588         4.400
14       14         2.225836         2.000
15       15         1.681606         1.800
16       16         4.080972         4.400
17       17         3.207432         3.600
18       18         2.922054         3.600
19       19         0.804373         1.200


In [22]:
# %% 7. การสร้างไฟล์ Submission

print("กำลังสร้างไฟล์ submission.csv...")

# เลือกเฉพาะคอลัมน์ที่ต้องการสำหรับไฟล์ submission
submission_df = test_df_featured[['pair_id']].copy()
submission_df['match_score'] = test_df_featured['predicted_score']

# ตรวจสอบว่าคะแนนไม่เกินขอบเขต (1-5)
# ในชุดข้อมูลนี้ ค่า cosine similarity ทำให้ score อยู่ในขอบเขต 0-5 อยู่แล้ว
# แต่อาจจำเป็นต้อง clip หากใช้ฟีเจอร์อื่น
submission_df['match_score'] = submission_df['match_score'].clip(0, 5)

print("\nตัวอย่างผลลัพธ์ในไฟล์ Submission:")
print(submission_df.head())

# บันทึกเป็นไฟล์ CSV
submission_df.to_csv("submission.csv", index=False)

print("\nสร้างไฟล์ submission.csv สำเร็จ!")


กำลังสร้างไฟล์ submission.csv...

ตัวอย่างผลลัพธ์ในไฟล์ Submission:
   pair_id  match_score
0        0     3.533265
1        1     3.408280
2        2     4.168404
3        3     3.634500
4        4     1.191888

สร้างไฟล์ submission.csv สำเร็จ!
